In [ ]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [ ]:
#############################
### Essentials functions ###
#############################
import FunctionMain as fm

In [ ]:
# ----------------- Parameters

k = 20
verbose = False

tps_max = 100
it_max = 5
it = 1
k_k = 3



In [ ]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# Parameters



# Interaction with the solver

data = []
file = './file_with_optimal_val/la03.txt'
n, m, data, T_machine, T_duration, duration, optimalval = fm.lire_fichier(file)


# --------- Call Solver constructor in Solver.py

solver = Solver(data)

model = CpoModel() 

# --------- Create the model variables 

solver.create_variables(model, n, m, duration)

# ------------ Solve the model

print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, k, n, m, T_machine, optimalval)

# ------------ Display the result

# for sol in msol:
#     sol.write()
print("Model solved !")


# ---------------- Interaction with the user



print("\n--------Interaction with the user...---------")

print("\nCreating the user...")
user = User()
print("User created !")

#Get the variables of the model
variables = solver.get_variables()

# ---------------- Interaction with the user
starts, matrix, list_indice, list_equal, list_obj, pref = fm.interactionUtilisateur(n, m, variables, msol, user, verbose)
fm.testOrderPreferences(user)   
fm.testDifferentSolutions(n, m, user)


In [ ]:
# print(list_equal)
# print(len(matrix))
# cl.clustering(layers, 120, 200, 1)
def abs_int(x,y):
    return max(x-y,y-x)
def abs_(L1,L2):
    L=[]
    for i in range(len(L1)):
        L.append(abs_int(L1[i],L2[i]))
    return L
def sum_(L):
    s=0
    for i in range(len(L)):
        s+=L[i]
    return s
def manhattan_distance(sol1, sol2):
    sol1np=np.array(sol1)
    sol2np=np.array(sol2)
    return sum_(abs_(sol1np,sol2np))
# def manhattan_distance_siala(self,sol1,sol2):
#     return np.sum(np.abs(sol1-sol2))

In [ ]:
############################################################
#### AJOUT DES CLUSTER A CHAQUE ITERATION
############################################################

import clustering as cl

###  -------------- Iteration of the solver with the preferences

# ----------------- Parameters

tps_max = 100
tps = runtime
it_max = 5
it = 1
k_k = 3

criterion = (tps < tps_max) and (it < it_max) 

# ----------------- Add the preferences to the model

while criterion :

    print("\n--------Iteration {}---------".format(it))

    it += 1
    
    solver = Solver(data)
    model = CpoModel() 

    # --------- Create the model variables 

    solver.create_variables(model, n, m, duration) 

    fm.constraintNewOldSolutionDifferent(n, m,  pref, model, solver)
    

# ------------ Solve the model

    print("\nSolving the model...")

    msol, nb_solution, runtime = solver.solve(model, k_k, n, m, T_machine, optimalval)

# ------------ Display the result

    # for sol in msol:
    #     sol.write()
    print("Model solved !")


# ---------------- Interaction with the user

    starts, matrix, list_indice, list_equal, list_obj, pref = fm.interactionUtilisateur(n, m, variables, msol, user, verbose)
    fm.testOrderPreferences(user)   
    fm.testDifferentSolutions(n, m, user)


#------------------Création des layers      
    layers = cl.create_layers(list_equal, matrix)
    print(layers)
    print(len(layers))

#------------------Définition du nombre de clusters par layer
    nb_clusters = fm.definirNbClusters(layers)

    
    for i in range(len(layers)):
        data = layers[i]
        print("-------------- Layer", i, "-----------------")
        if(len(data)==1):
            solver.add_constraint(manhattan_distance(data[0], [ model.start_of(variables[i//m][i%m]) for i in range(n*m)]) >=3)
            continue
        
        k = nb_clusters[i]

        k , leaves , labels, runtime = cl.my_agglo_k(data, k, 'single')

        avg=[[0 for i in range(len(data[0]))] for i in range(k)]
        card=[0 for i in range(k) ]

        for i in range(len(data)):
            card[labels[i]]+=1
            avg[labels[i]]=[avg[labels[i]][k]+data[i][k] for k in range(len(data[i]))]
        
        for i in range(k):
            avg[i]=[avg[i][k]/card[i] for k in range(len(data[0]))]
        
        the_ones=[None for i in range(k)]
        for i in range(len(data)):
            if(the_ones[labels[i]]==None or manhattan_distance(data[i],avg[labels[i]])<manhattan_distance(the_ones[labels[i]],avg[labels[i]])):
                    the_ones[labels[i]]=data[i]
        
        #ajout de contraintes, les solutions doivent etre distantes des centres de clusters, en chaque layer , d'une distance supérieure à 3
        for i in range(k):
            solver.add_constraint(manhattan_distance(the_ones[i],[ model.start_of(variables[i//m][i%m]) for i in range(n*m)]) >=3)
    

#------------------Condition d'arrêt
    tps += runtime

    criterion = (tps < tps_max) and (it < it_max)

    fm.stopCondition(it, it_max, tps, tps_max)